## Extract

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Dependencies
import pandas as pd
import requests
from census import Census
from citipy import citipy
import gmaps
import os
import pymongo

# Census API Key
from config import (census_api_key, g_key)
c = Census(census_api_key, year=2019)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/datamade/census for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01002_001E", "B25077_001E","B01003_001E","B15003_022E","B15003_023E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B19013_001E": "MedianHouseholdIncome",
                                      "B01002_001E": "MedianAge",
                                      "B25077_001E": "MedianHomeValue",
                                      "B01003_001E": "Population",
                                      "B15003_022E": "Number of People who hold a Bachelors",
                                      "B15003_023E": "Number of People who hold a Masters",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Bachelors Rate (Number of People who hold a Bachelors / Population)
census_pd["BachelorsRate"] = 100 * \
    census_pd["Number of People who hold a Bachelors"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Masters Rate (Number of People who hold a Masters / Population)
census_pd["MastersRate"] = 100 * \
    census_pd["Number of People who hold a Masters"].astype(
        int) / census_pd["Population"].astype(int)


# Final DataFrame
census_pd = census_pd[["Zipcode", "MedianHouseholdIncome", "MedianAge", "MedianHomeValue","Population",
                       "BachelorsRate","MastersRate"]]

# Visualize
print(len(census_pd))
# census_pd.to_csv("Resources/census_data.csv", encoding="utf-8", index=False)
census_pd.head()

33120


,Zipcode,MedianHouseholdIncome,MedianAge,MedianHomeValue,Population,BachelorsRate,MastersRate
0,00601,14361.0,41.9,83900.0,17113.0,10.103430,0.905744
1,00602,16807.0,42.9,85300.0,37751.0,10.598395,2.977405
2,00603,16049.0,42.1,118400.0,47081.0,11.811559,3.355919
3,00606,12119.0,44.3,80800.0,6392.0,3.613892,1.814768
4,00610,19898.0,42.7,87600.0,26686.0,10.488646,2.431987


In [3]:
# Add columns for city, latitude and longitude
census_pd["City"] = ""
census_pd["State"] = ""
census_pd["Lat"] = ""
census_pd["Lng"] = ""
census_pd.head()

,Zipcode,MedianHouseholdIncome,MedianAge,MedianHomeValue,Population,BachelorsRate,MastersRate,City,State,Lat,Lng
0,00601,14361.0,41.9,83900.0,17113.0,10.103430,0.905744,,,,
1,00602,16807.0,42.9,85300.0,37751.0,10.598395,2.977405,,,,
2,00603,16049.0,42.1,118400.0,47081.0,11.811559,3.355919,,,,
3,00606,12119.0,44.3,80800.0,6392.0,3.613892,1.814768,,,,
4,00610,19898.0,42.7,87600.0,26686.0,10.488646,2.431987,,,,


In [4]:
# filtering census data to get the Portland zipcodes
portland_census_pd = census_pd.loc[(census_pd["Zipcode"]>="97080") & (census_pd["Zipcode"]<="97267")]
portland_census_pd

,Zipcode,MedianHouseholdIncome,MedianAge,MedianHomeValue,Population,BachelorsRate,MastersRate,City,State,Lat,Lng
31924,97080,75431.0,39.1,336900.0,44335.0,12.626593,5.171986,,,,
31925,97086,94031.0,39.8,468300.0,30858.0,19.602696,8.451617,,,,
31926,97089,96744.0,42.4,433000.0,13765.0,13.745005,5.521250,,,,
31927,97101,76397.0,40.8,354300.0,4278.0,8.976157,4.534829,,,,
31928,97102,122708.0,65.4,638500.0,318.0,29.245283,23.584906,,,,
...,...,...,...,...,...,...,...,...,...,...,...
31997,97233,42452.0,32.6,252700.0,40477.0,5.998468,1.531734,,,,
31998,97236,50663.0,35.4,282600.0,40892.0,7.830382,3.318498,,,,
31999,97239,93459.0,39.5,593500.0,18190.0,29.059923,16.234195,,,,
32000,97266,53984.0,36.9,274700.0,35727.0,12.256837,3.761861,,,,


In [ ]:
# create a params dict that will be updated with new city each iteration
params = {"key": g_key}
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

count = 0
# Loop through the filtered_small_census_pd and run a lat/long search for each city
for index, row in portland_census_pd.iterrows():
    
#     if count == 15:
#         break

    zipcode = row['Zipcode']

    # update address key value
    params['address'] = f"{zipcode}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
#     convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        portland_census_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        portland_census_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
        portland_census_pd.loc[index, "City"] = cities_lat_lng["results"][0]["address_components"][1]["long_name"]
        portland_census_pd.loc[index, "State"] = cities_lat_lng["results"][0]["address_components"][3]["short_name"]
#         print(f"count {count}")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    count = count + 1

# Print to csv
# portland_census_pd.to_csv("portland_census_data.csv", encoding="utf-8", index=False)
    
# Visualize to confirm lat lng city appear
portland_census_pd

## Transform

In [ ]:
# Find all State values that do not equal "OR"
portland_census_pd[(portland_census_pd['State'] != 'OR')]

In [ ]:
# Count number of rows
len(portland_census_pd["Zipcode"].unique())

In [ ]:
# Drop zipcode '97208'
bad_zipcode = ["97208"]
portland_census_pd = portland_census_pd[portland_census_pd["Zipcode"].isin(bad_zipcode)==False]
portland_census_pd

In [ ]:
# Count number of rows
len(portland_census_pd["Zipcode"].unique())

In [ ]:
# Change all State values to OR
portland_census_pd["State"].replace({"US": "OR", "Multnomah County": "OR"}, inplace=True)
portland_census_pd = portland_census_pd.loc[portland_census_pd["City"] == "Portland"]
portland_census_pd.head()

## Load

In [ ]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# create mongo database
db = client.portland_census_db

# Declare the collection, drop any preexisting
db.portland_census_db.drop()
facts = db.portland_census_db

In [ ]:
# Loop through dataframe and load to Mongo Database
for index,row in portland_census_pd.iterrows():
    post = {"Zipcode": portland_census_pd.loc[index,"Zipcode"],
            "MedianHouseholdIncome": portland_census_pd.loc[index,"MedianHouseholdIncome"],
            "MedianAge": portland_census_pd.loc[index,"MedianAge"],
            "MedianHomeValue": portland_census_pd.loc[index,"MedianHomeValue"],
            "Population": portland_census_pd.loc[index,"Population"],
            "BachelorsRate": portland_census_pd.loc[index,"BachelorsRate"],
            "MastersRate": portland_census_pd.loc[index,"MastersRate"],
            "City": portland_census_pd.loc[index,"City"],
            "State": portland_census_pd.loc[index,"State"],
            "Lat": portland_census_pd.loc[index,"Lat"],
            "Lng": portland_census_pd.loc[index,"Lng"],
           }
    facts.insert_one(post)